In [ ]:
pip install torchvision


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define the FeatureExtractor
class FeatureExtractor(nn.Module):
    def __init__(self, weights_path='feature_extractor.pth'):
        super(FeatureExtractor, self).__init__()
        # Define the architecture
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Load pre-trained weights on CPU
        state_dict = torch.load(weights_path, map_location=torch.device('cpu'))
        self.load_state_dict(state_dict)

    def forward(self, x):
        x = self.features(x)
        return x

# Create an instance of the FeatureExtractor
feature_extractor = FeatureExtractor()

# Define the Multi-Task Model
class MultiTaskModel(nn.Module):
    def __init__(self, feature_extractor):
        super(MultiTaskModel, self).__init__()
        self.feature_extractor = feature_extractor
        self.digit_classifier = nn.Linear(64 * 7 * 7, 10)  # Adjust the input size based on feature extractor's output
        self.binary_classifier = nn.Linear(64 * 7 * 7, 2)  # Adjust the input size based on feature extractor's output

    def forward(self, x):
        features = self.feature_extractor(x)
        features = features.view(features.size(0), -1)  # Flatten the features
        digit_logits = self.digit_classifier(features)
        binary_logits = self.binary_classifier(features)
        return digit_logits, binary_logits

# Create an instance of the MultiTaskModel
model = MultiTaskModel(feature_extractor)

# Define the loss functions and optimizer
criterion_digit = nn.CrossEntropyLoss()
criterion_binary = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Load the MNIST dataset and preprocess it
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST('data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Training
epochs = 10
for epoch in range(epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        digit_logits, binary_logits = model(data)
        loss_digit = criterion_digit(digit_logits, target)
        binary_target = (target > 5).long()
        loss_binary = criterion_binary(binary_logits, binary_target)
        loss = loss_digit + loss_binary
        loss.backward()
        optimizer.step()

# Evaluation
model.eval()
correct_digit, correct_binary = 0, 0
total = 0
with torch.no_grad():
    for batch_idx, (data, target) in enumerate(train_loader):
        digit_logits, binary_logits = model(data)
        _, predicted_digit = digit_logits.max(1)
        _, predicted_binary = binary_logits.max(1)
        correct_digit += predicted_digit.eq(target).sum().item()
        correct_binary += predicted_binary.eq((target > 5).long()).sum().item()
        total += target.size(0)

digit_accuracy = 100.0 * correct_digit / total
binary_accuracy = 100.0 * correct_binary / total

print(f"Digit Classification Accuracy: {digit_accuracy:.2f}%")
print(f"Binary Classification Accuracy: {binary_accuracy:.2f}%")
